# Classical NN (MLP)


**ONNX conversion**

In [4]:
!pip install onnx tf2onnx
!pip install tensorflow_decision_forests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successf

In [92]:
# libraries for data processing and machine learning
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import tensorflow as tf
import tensorflow_decision_forests as tfdf

#import skl2onnx
#from skl2onnx import convert_sklearn
#from skl2onnx.common.data_types import FloatTensorType


# dataset load
dataset = pd.read_csv("dataset_web_spoof_output.csv")


**Pre-Process If Needed**

In [109]:
dataset.dropna()
dataset.reset_index(drop=True)
dataset['wlan.seq'] = dataset['wlan.seq'].astype(int)
dataset.isnull().sum()



Label                         0
frame.encap_type              0
frame.len                     0
frame.number                  0
frame.time_delta              0
frame.time_delta_displayed    0
frame.time_epoch              0
frame.time_relative           0
wlan.duration                 0
wlan.fc.frag                  0
wlan.fc.order                 0
wlan.fc.moredata              0
wlan.fc.protected             0
wlan.fc.pwrmgt                0
wlan.fc.type                  0
wlan.fc.retry                 0
wlan.fc.subtype               0
wlan.seq                      0
dtype: int64

In [97]:
#dataset.head()

#dataset['Label'] = dataset['Label'].replace({'0': 0, '1': 1, 'Kr00k': 1})
dataset['Label'] = dataset['Label'].replace({'0': 0, '1': 1, 'Web_Spoofing': 1})

dataset['Label'] = dataset['Label'].astype(int)

print(dataset['Label'].unique())

[0 1]


In [98]:
x = dataset.drop(columns=['Label'])


In [99]:
y = dataset['Label']

In [100]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [101]:
import tensorflow as tf


model = tf.keras.models.Sequential()

In [102]:
model.add(tf.keras.layers.Dense(256, input_shape=(x_train.shape[1],), activation='relu'))
#model.add(tf.keras.layers.Dense(256,  activation='sigmoid'))
model.add(tf.keras.layers.Dense(1,  activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [103]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.AUC()])


In [104]:
dataset.isnull().sum()


Label                         0
frame.encap_type              0
frame.len                     0
frame.number                  0
frame.time_delta              0
frame.time_delta_displayed    0
frame.time_epoch              0
frame.time_relative           0
wlan.duration                 0
wlan.fc.frag                  0
wlan.fc.order                 0
wlan.fc.moredata              0
wlan.fc.protected             0
wlan.fc.pwrmgt                0
wlan.fc.type                  0
wlan.fc.retry                 0
wlan.fc.subtype               0
wlan.seq                      0
dtype: int64

In [105]:
dataset.describe(include='all')
dataset.dtypes

#dataset['Label'].value_counts()



Label                           int64
frame.encap_type                int64
frame.len                       int64
frame.number                    int64
frame.time_delta              float64
frame.time_delta_displayed    float64
frame.time_epoch                int64
frame.time_relative           float64
wlan.duration                   int64
wlan.fc.frag                    int64
wlan.fc.order                   int64
wlan.fc.moredata                int64
wlan.fc.protected               int64
wlan.fc.pwrmgt                  int64
wlan.fc.type                    int64
wlan.fc.retry                   int64
wlan.fc.subtype                 int64
wlan.seq                        int64
dtype: object

In [107]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test)) #DNN


Epoch 1/10
58375/58375 ━━━━━━━━━━━━━━━━━━━━ 165s 3ms/step - accuracy: 0.7689 - auc_6: 0.5530 - loss: 189961.4844 - precision_6: 0.2423 - recall_6: 0.2424 - val_accuracy: 0.8487 - val_auc_6: 0.5000 - val_loss: 46908.7422 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00
Epoch 2/10
58375/58375 ━━━━━━━━━━━━━━━━━━━━ 159s 3ms/step - accuracy: 0.7774 - auc_6: 0.5643 - loss: 25650.4297 - precision_6: 0.2472 - recall_6: 0.2267 - val_accuracy: 0.8487 - val_auc_6: 0.5000 - val_loss: 0.4250 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00
Epoch 3/10
58375/58375 ━━━━━━━━━━━━━━━━━━━━ 203s 3ms/step - accuracy: 0.8476 - auc_6: 0.4998 - loss: 0.4274 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_accuracy: 0.8487 - val_auc_6: 0.5000 - val_loss: 0.4251 - val_precision_6: 0.0000e+00 - val_recall_6: 0.0000e+00
Epoch 4/10
58375/58375 ━━━━━━━━━━━━━━━━━━━━ 201s 3ms/step - accuracy: 0.8480 - auc_6: 0.5006 - loss: 0.4322 - precision_6: 0.0000e+00 - recall_6: 0.0000e+00 - val_accuracy: 0.

In [108]:
#from tensorflow.keras.utils import plot_model
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 256)                 │           4,608 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,597 (57.02 KB)

 Trainable params: 4,865 (19.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,732 (38.02 KB)

# Evaluation on new labeled data

In [ ]:
# test data (from AWID3)
new_data = pd.read_csv("deauth_wlan_frame.csv")

# Drop label
x_new = new_data.drop(columns=['Label'])
y_new = new_data['Label']

# Evaluate the model on the new data
loss, accuracy, precision, recall, auc = model.evaluate(x_new, y_new)

print("Evaluation Metrics:")
print("Loss:", loss)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("AUC:", auc)

# Predict labels for the new data
predicted_labels = model.predict(x_new)

# Convert predicted probabilities to binary label, this will be implemented in the IDS
predicted_labels_binary = (predicted_labels > 0.5).astype(int)

new_data['Predicted_Label'] = predicted_labels_binary

packets_labeled_1 = new_data[new_data['Label'] == 1]
packets_labeled_1.to_csv("packets_labeled_as_1.csv", index=False)

1563/1563 [==============================] - 5s 3ms/step - loss: 0.2642 - accuracy: 0.9333 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - auc_1: 0.5000
Evaluation Metrics:
Loss: 0.26424455642700195
Accuracy: 0.9332799911499023
Precision: 0.0
Recall: 0.0
AUC: 0.5
1563/1563 [==============================] - 3s 2ms/step


In [ ]:
dataset.describe()

,Label,frame.encap_type,frame.len,frame.number,frame.time_delta,frame.time_delta_displayed,frame.time_epoch,frame.time_relative,wlan.duration,wlan.fc.frag,wlan.fc.order,wlan.fc.moredata,wlan.fc.protected,wlan.fc.pwrmgt,wlan.fc.type,wlan.fc.retry,wlan.fc.subtype
count,2.900420e+06,2900420.0,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06,2.900420e+06
mean,6.612939e-02,23.0,4.425136e+02,1.450224e+06,2.076858e-04,2.076858e-04,1.609009e+09,3.871253e+02,1.712853e+02,1.827322e-05,2.172099e-05,1.126389e-03,3.834986e-01,7.419615e-04,1.195249e+00,2.976638e-02,1.016192e+01
std,2.485082e-01,0.0,6.346119e+02,8.372906e+05,2.289991e-03,2.289991e-03,1.207750e+02,1.207735e+02,4.822934e+02,4.274680e-03,4.660529e-03,3.354281e-02,4.862381e-01,2.722887e-02,6.500994e-01,1.699422e-01,2.064694e+00
min,0.000000e+00,23.0,7.000000e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.609009e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,23.0,7.000000e+01,7.251098e+05,6.000000e-06,6.000000e-06,1.609009e+09,2.715330e+02,4.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,8.000000e+00
50%,0.000000e+00,23.0,8.800000e+01,1.450220e+06,5.900000e-05,5.900000e-05,1.609009e+09,3.896722e+02,4.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+01
75%,0.000000e+00,23.0,2.580000e+02,2.175331e+06,1.290000e-04,1.290000e-04,1.609009e+09,4.914989e+02,1.560000e+02,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,1.200000e+01
max,1.000000e+00,23.0,3.248000e+03,2.900458e+06,2.048170e-01,2.048170e-01,1.609010e+09,6.023876e+02,3.000000e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.400000e+01


In [ ]:
dataset.columns

Index(['Label', 'frame.encap_type', 'frame.len', 'frame.number',
       'frame.time_delta', 'frame.time_delta_displayed', 'frame.time_epoch',
       'frame.time_relative', 'wlan.duration', 'wlan.fc.frag', 'wlan.fc.order',
       'wlan.fc.moredata', 'wlan.fc.protected', 'wlan.fc.pwrmgt',
       'wlan.fc.type', 'wlan.fc.retry', 'wlan.fc.subtype'],
      dtype='object')

In [ ]:
result = pd.read_csv("deauth_wlan_frame.csv")

In [ ]:
result[result['Label'] == 1]['wlan.fc.subtype'].value_counts()

NameError: name 'result' is not defined

In [ ]:
result[(result['Label'] != 1)&(result['wlan.fc.subtype']== 12)].count() #nu coreleaza

Label                         10020
frame.encap_type              10020
frame.len                     10020
frame.number                  10020
frame.time_delta              10020
frame.time_delta_displayed    10020
frame.time_epoch              10020
frame.time_relative           10020
wlan.duration                 10020
wlan.fc.frag                  10020
wlan.fc.order                 10020
wlan.fc.moredata              10020
wlan.fc.protected             10020
wlan.fc.pwrmgt                10020
wlan.fc.type                  10020
wlan.fc.retry                 10020
wlan.fc.subtype               10020
dtype: int64

In [ ]:
result[(result['Label'] == 1)&(result['wlan.fc.subtype']== 12)].count() #nu coreleaza

Label                         3336
frame.encap_type              3336
frame.len                     3336
frame.number                  3336
frame.time_delta              3336
frame.time_delta_displayed    3336
frame.time_epoch              3336
frame.time_relative           3336
wlan.duration                 3336
wlan.fc.frag                  3336
wlan.fc.order                 3336
wlan.fc.moredata              3336
wlan.fc.protected             3336
wlan.fc.pwrmgt                3336
wlan.fc.type                  3336
wlan.fc.retry                 3336
wlan.fc.subtype               3336
dtype: int64

# Prediction for new unlabeled data

In [ ]:
real_data = pd.read_csv("real_data_deauth.csv")

In [ ]:
real_data.replace({False: 0, True: 1}, inplace=True) # convert bools to int

In [ ]:
real_data.dtypes


frame.encap_type                int64
frame.len                       int64
frame.number                    int64
frame.time_delta              float64
frame.time_delta_displayed    float64
frame.time_epoch              float64
frame.time_relative           float64
wlan.duration                   int64
wlan.fc.frag                    int64
wlan.fc.order                   int64
wlan.fc.moredata                int64
wlan.fc.protected               int64
wlan.fc.pwrmgt                  int64
wlan.fc.type                    int64
wlan.fc.retry                   int64
wlan.fc.subtype                 int64
dtype: object

In [ ]:
dataset.dtypes

Label                           int64
frame.encap_type                int64
frame.len                       int64
frame.number                    int64
frame.time_delta              float64
frame.time_delta_displayed    float64
frame.time_epoch              float64
frame.time_relative           float64
wlan.duration                   int64
wlan.fc.frag                    int64
wlan.fc.order                   int64
wlan.fc.moredata                int64
wlan.fc.protected               int64
wlan.fc.pwrmgt                  int64
wlan.fc.type                    int64
wlan.fc.retry                   int64
wlan.fc.subtype                 int64
dtype: object

In [ ]:
real_results = model.predict(real_data)

187/187 [==============================] - 0s 2ms/step


In [ ]:
pd.DataFrame(real_results).describe()

,0
count,5968.000000
mean,0.027152
std,0.000221
min,0.027117
25%,0.027117
50%,0.027117
75%,0.027117
max,0.028587


In [ ]:
real_data['predicted label'] = real_results

In [ ]:
pd.DataFrame(real_results).describe()

,0
count,5968.000000
mean,0.027152
std,0.000221
min,0.027117
25%,0.027117
50%,0.027117
75%,0.027117
max,0.028587


In [ ]:
real_data.head()

,frame.encap_type,frame.len,frame.number,frame.time_delta,frame.time_delta_displayed,frame.time_epoch,frame.time_relative,wlan.duration,wlan.fc.frag,wlan.fc.order,wlan.fc.moredata,wlan.fc.protected,wlan.fc.pwrmgt,wlan.fc.type,wlan.fc.retry,wlan.fc.subtype,predicted label
0,20,348,1,0.000000,0.000000,2.000539,0.000000,0,0,0,0,0,0,0,0,8,0.028587
1,20,263,2,-0.000184,-0.000184,2.000355,-0.000184,0,0,0,0,0,0,0,0,8,0.028587
2,20,117,3,-0.000115,-0.000115,2.000240,-0.000299,0,0,0,0,0,0,0,0,8,0.028587
3,20,213,4,0.000546,0.000546,2.000786,0.000247,0,0,0,0,0,0,0,0,8,0.028587
4,20,265,5,-0.000347,-0.000347,2.000439,-0.000100,0,0,0,0,0,0,0,0,8,0.028587


In [ ]:
real_data[real_data['predicted label'] > 0.1]

NameError: name 'real_data' is not defined

## Save model as ONNX for later usage

In [51]:
import tf2onnx
import onnx

In [119]:
input_signature = [tf.TensorSpec([None, x_train.shape[1]], tf.float32, name='input')]
model.output_names=['output']


In [120]:
print(input_signature)
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=17)


Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'


[TensorSpec(shape=(None, 17), dtype=tf.float32, name='input')]


In [121]:
onnx.save(onnx_model, "kr00k.onnx")